# Challenge - NLP - 5 - Convolutional Sequence to Sequence Learning
## Steps of problem solving
First step was preparing an existing solution.
1. Model analysis. I read "Convolutional Sequence to Sequence Learning" paper. I already worked with it when I was studied NLP in Samsung AI Academy. I checked code correctness of convolutional layers, seq-to-seq model.
2. Filling the gaps. I added tokenizer and dataset iterator.
3. Bug fixes like updating versions of libraries and other little mistakes.
Second step was model optimization.
Model had inappropriate parameters and terrible perplexity.
I used the parameters of source paper excepting optimizer.

## Model parameters
After analysis of paper I got few sets of optimal model parameters and started experiments.
I worked with 10,15,20 layers, 256,512 hidden units, kernel sizes 3/3, 3/5, Nesterov accelerated gradient (parameter set from paper "model optimization" momentum 0.99, learning rate 0.25) and Adam optimizator. Batch sizes 48,64,128. I tested not all combinations of these parameters, but used combinations from paper and tried to improve them using individual parameters from other hyperparameters sets to find best solution.

## Best found parameters (9.933 PPL)

Best result shows this parameters set from paper:
*Our encoder has 15 layers and the decoder has 15 layers, both with 512 hidden
units in the first ten layers and 768 units in the subsequent
three layers, all using kernel width 3.*

In combination with these batch size:
*Unless otherwise stated, we use mini-batches of 64 sentences.*

Despite the fact that the paper uses the Nesterov accelerated gradient (SGD) optimizer in my final solution I use Adam optimizer.

## Comparison of Adam and Nesterov optimizer on final parameters sets

Adam validation of perplex on 1 epoch = 25.025
Test perplex on 5, 10, 15 epoch = 10.513, 9.279, 9.087
After 15 epoch we need to stop, because then we can see growing of validation PPL from 3.975 -> 5.064 -> 10.210 -> 5.957 -> 65141 -> ~num*10^174 (loss explosion)

But using Nesterov optimizer we can see better start on validation 34.764 PPL, no increasing loss and explosion loss on late train stages, but it learns very slowly and after 15 epochs on test shows only 21.465 PPL.

## Model improvement ideas
Results can be improved using
1. bigger dataset (WMT for example)
2. parameters difference in layers (as mentiones in paper: special parameters in first 10 layers and other in 3 last)
3. data preprocessing - cleaning improvements/tests
4. using SGD optimizer with Nesterov accelerated gradient. Yes, Adam shows better result. It is golden hammer, but can exist better result with Nesterov.
5. using other models, Transformers for example:)
6. keep optimizing hyperparameters (need more time and computing power)

sergeev46v@gmail.com - additional information

In [1]:
#!pip install -U torch==1.7.1
!pip install -U torchtext==0.11.0

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.datasets import Multi30k
from torchtext.datasets import IWSLT2017
from torchtext.legacy.data import Field, BucketIterator, TabularDataset

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
!python -m spacy download en
!python -m spacy download de

     |████████████████████████████████| 12.0 MB 8.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9 MB 10.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


Load the German and English spaCy models.

In [5]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

We create the tokenizers.

In [6]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [7]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

Then, we load our dataset.

In [8]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), 
                                                    fields=(SRC, TRG))

In [9]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
device

device(type='cuda')

In [12]:
BATCH_SIZE = 64

#Create an iterator below (using BucketIterator) to create your train, validation and test data.
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device)

In [13]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 emb_dim, 
                 hid_dim, 
                 n_layers, 
                 kernel_size, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        assert kernel_size % 2 == 1, "Kernel size must be odd!"
        
        self.device = device
        
        self.scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)
        
        self.tok_embedding = nn.Embedding(input_dim, emb_dim)
        self.pos_embedding = nn.Embedding(max_length, emb_dim)
        
        self.emb2hid = nn.Linear(emb_dim, hid_dim)
        self.hid2emb = nn.Linear(hid_dim, emb_dim)
        
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = hid_dim, 
                                              out_channels = 2 * hid_dim, 
                                              kernel_size = kernel_size, 
                                              padding = (kernel_size - 1) // 2)
                                    for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
               
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        #create position tensor
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        
        #embed tokens and positions
        tok_embedded = self.tok_embedding(src)
        pos_embedded = self.pos_embedding(pos)
        
        
        #combine embeddings by elementwise summing
        embedded = self.dropout(tok_embedded + pos_embedded)
        
        #pass embedded through linear layer to convert from emb dim to hid dim
        conv_input = self.emb2hid(embedded)
        
        #permute for convolutional layer
        conv_input = conv_input.permute(0, 2, 1) 
        
        #begin convolutional blocks...
        
        for i, conv in enumerate(self.convs):
        
            #pass through convolutional layer
            conved = conv(self.dropout(conv_input))

            #conved = [batch size, 2 * hid dim, src len]

            #pass through GLU activation function
            conved = F.glu(conved, dim = 1)

            #conved = [batch size, hid dim, src len]
            
            #apply residual connection
            conved = (conved + conv_input) * self.scale

            #conved = [batch size, hid dim, src len]
            
            #set conv_input to conved for next loop iteration
            conv_input = conved
        
        #...end convolutional blocks
        
        #permute and convert back to emb dim
        conved = self.hid2emb(conved.permute(0, 2, 1))
        
        #conved = [batch size, src len, emb dim]
        
        #elementwise sum output (conved) and input (embedded) to be used for attention
        combined = (conved + embedded) * self.scale
        
        #combined = [batch size, src len, emb dim]
        
        return conved, combined

In [14]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 emb_dim, 
                 hid_dim, 
                 n_layers, 
                 kernel_size, 
                 dropout, 
                 trg_pad_idx, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.kernel_size = kernel_size
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
        self.scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)
        
        self.tok_embedding = nn.Embedding(output_dim, emb_dim)
        self.pos_embedding = nn.Embedding(max_length, emb_dim)
        
        self.emb2hid = nn.Linear(emb_dim, hid_dim)
        self.hid2emb = nn.Linear(hid_dim, emb_dim)
        
        self.attn_hid2emb = nn.Linear(hid_dim, emb_dim)
        self.attn_emb2hid = nn.Linear(emb_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim, output_dim)
        
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = hid_dim, 
                                              out_channels = 2 * hid_dim, 
                                              kernel_size = kernel_size)
                                    for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
      
    def calculate_attention(self, embedded, conved, encoder_conved, encoder_combined):
                
        #permute and convert back to emb dim
        conved_emb = self.attn_hid2emb(conved.permute(0, 2, 1))
        
        combined = (conved_emb + embedded) * self.scale
        
        energy = torch.matmul(combined, encoder_conved.permute(0, 2, 1))
        
        attention = F.softmax(energy, dim=2)
        
        attended_encoding = torch.matmul(attention, encoder_combined)
        
        #convert from emb dim -> hid dim
        attended_encoding = self.attn_emb2hid(attended_encoding)
        
        #apply residual connection
        attended_combined = (conved + attended_encoding.permute(0, 2, 1)) * self.scale
        
        #attended_combined = [batch size, hid dim, trg len]
        
        return attention, attended_combined
        
    def forward(self, trg, encoder_conved, encoder_combined):
        
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
            
        #create position tensor
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #embed tokens and positions
        tok_embedded = self.tok_embedding(trg)
        pos_embedded = self.pos_embedding(pos)
        
        #combine embeddings by elementwise summing
        embedded = self.dropout(tok_embedded + pos_embedded)
        
        #pass embedded through linear layer to go through emb dim -> hid dim
        conv_input = self.emb2hid(embedded)
        
        #permute for convolutional layer
        conv_input = conv_input.permute(0, 2, 1) 
        
        batch_size = conv_input.shape[0]
        hid_dim = conv_input.shape[1]
        
        for i, conv in enumerate(self.convs):
        
            #apply dropout
            conv_input = self.dropout(conv_input)
        
            #need to pad so decoder can't "cheat"
            padding = torch.zeros(batch_size, 
                                  hid_dim, 
                                  self.kernel_size - 1).fill_(self.trg_pad_idx).to(self.device)
                
            padded_conv_input = torch.cat((padding, conv_input), dim = 2)
        
            #pass through convolutional layer
            conved = conv(padded_conv_input)
    
            #pass through GLU activation function
            conved = F.glu(conved, dim = 1)

            #conved = [batch size, hid dim, trg len]
            
            #calculate attention
            attention, conved = self.calculate_attention(embedded, 
                                                         conved, 
                                                         encoder_conved, 
                                                         encoder_combined)
            
            #attention = [batch size, trg len, src len]
            
            #apply residual connection
            conved = (conved + conv_input) * self.scale
            
            #set conv_input to conved for next loop iteration
            conv_input = conved
            
        conved = self.hid2emb(conved.permute(0, 2, 1))
            
        output = self.fc_out(self.dropout(conved))
            
        return output, attention

In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg):
        
        
        #calculate z^u (encoder_conved) and (z^u + e) (encoder_combined)
        #encoder_conved is output from final encoder conv. block
        #encoder_combined is encoder_conved plus (elementwise) src embedding plus 
        #  positional embeddings 
        encoder_conved, encoder_combined = self.encoder(src)
        
        #calculate predictions of next words
        #output is a batch of predictions for each word in the trg sentence
        #attention a batch of attention scores across the src sentence for 
        #  each word in the trg sentence
        output, attention = self.decoder(trg, encoder_conved, encoder_combined)
        
        #output = [batch size, trg len - 1, output dim]
        #attention = [batch size, trg len - 1, src len]
        
        return output, attention

In [16]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
EMB_DIM = 256
HID_DIM = 512 # each conv. layer has 2 * hid_dim filters
ENC_LAYERS = 15 # number of conv. blocks in encoder
DEC_LAYERS = 15 # number of conv. blocks in decoder
ENC_KERNEL_SIZE = 3 # must be odd!
DEC_KERNEL_SIZE = 3 # can be even or odd
ENC_DROPOUT = 0.25
DEC_DROPOUT = 0.25
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
    
enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, ENC_LAYERS, ENC_KERNEL_SIZE, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, DEC_LAYERS, DEC_KERNEL_SIZE, DEC_DROPOUT, TRG_PAD_IDX, device)

model = Seq2Seq(enc, dec).to(device)

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 53,090,565 trainable parameters


In [18]:
optimizer = optim.Adam(model.parameters())

In [19]:
#optimizer = torch.optim.SGD(model.parameters(), lr=0.25, momentum=0.99, nesterov=True)

In [20]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [21]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
        
        output_dim = output.shape[-1]
        
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [22]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])

            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [23]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [24]:
N_EPOCHS = 25
CLIP = 0.1

best_valid_loss = float('inf')

#model.load_state_dict(torch.load('tut5-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, train_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')

    if train_loss < 100000 and valid_loss < 10000:
      print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    else:
      print('PPL explosion!')
      break

    if (epoch+1) % 5 == 0:
      test_loss = evaluate(model, test_iterator, criterion)### The model should achieve a perplexity of 30 or lower on the test data.
      if test_loss < 100000:
        print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')
      else:
        break


Epoch: 01 | Time: 1m 22s
	Train Loss: 6.771 | Train PPL: 872.267
	 Val. Loss: 4.837 |  Val. PPL: 126.091
Epoch: 02 | Time: 1m 22s
	Train Loss: 2.894 | Train PPL:  18.062
	 Val. Loss: 2.554 |  Val. PPL:  12.862
Epoch: 03 | Time: 1m 21s
	Train Loss: 2.494 | Train PPL:  12.110
	 Val. Loss: 2.274 |  Val. PPL:   9.715
Epoch: 04 | Time: 1m 21s
	Train Loss: 2.267 | Train PPL:   9.647
	 Val. Loss: 2.083 |  Val. PPL:   8.032
Epoch: 05 | Time: 1m 21s
	Train Loss: 2.112 | Train PPL:   8.267
	 Val. Loss: 1.949 |  Val. PPL:   7.023
| Test Loss: 2.467 | Test PPL:  11.792 |
Epoch: 06 | Time: 1m 21s
	Train Loss: 1.994 | Train PPL:   7.347
	 Val. Loss: 1.844 |  Val. PPL:   6.320
Epoch: 07 | Time: 1m 21s
	Train Loss: 1.899 | Train PPL:   6.677
	 Val. Loss: 1.764 |  Val. PPL:   5.837
Epoch: 08 | Time: 1m 21s
	Train Loss: 1.826 | Train PPL:   6.207
	 Val. Loss: 1.692 |  Val. PPL:   5.432
Epoch: 09 | Time: 1m 21s
	Train Loss: 1.760 | Train PPL:   5.810
	 Val. Loss: 1.637 |  Val. PPL:   5.141
Epoch: 10 | Ti

In [25]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)### The model should achieve a perplexity of 30 or lower on the test data.
print(f'| Test Loss: {test_loss:.3f}')
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 2.296
| Test Loss: 2.296 | Test PPL:   9.933 |
